In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from collections import Counter
from sklearn.metrics import log_loss
#import xgboost as xgb
import pickle

In [2]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "recommendation_paths", "bankproduct_recommendation")

Extracting : 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


In [3]:
df= pd.read_csv('sliced.csv')

C:\Users\Fahad.A\AppData\Local\Temp\ipykernel_17272\3488184092.py:1: DtypeWarning: Columns (10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('sliced.csv')


In [3]:
df['Status_Dt'] = pd.to_datetime(df['Status_Dt'],format="%Y-%m-%d")

In [4]:
df['Year'] = df['Status_Dt'].dt.year
df['Monthly'] = df['Status_Dt'].dt.month
df['Day'] = df['Status_Dt'].dt.day

In [5]:
numeric_attribs = ['Age', 'Is_New_Cust', 'Cust_Since', 'Cust_Active_Status']
for attrib in numeric_attribs:
    #errors='coerce' - when value is invalid, set it to NAN
    df[attrib] = pd.to_numeric(df[attrib], errors='coerce')

In [6]:
df.drop('Is_Spouse_Emp', axis=1, inplace=True)
df.drop('Lst_Dt_Primary_Cust', axis=1, inplace=True)
df.drop('Address_Type', axis=1, inplace=True)
df.drop('Join_Dt', axis=1, inplace=True)
df.drop('Status_Dt', axis=1, inplace=True)
df.drop('Cust_ID', axis=1, inplace=True)
df.drop('Deceased_Flg', axis=1, inplace=True)
df.drop('Cust_Type_Mth_Beg', axis=1, inplace=True)
df.drop('Relship_Status', axis=1, inplace=True)
df.drop('Day', axis=1, inplace=True)
df.drop('Year', axis=1, inplace=True)
df.drop('Cust_Ctry', axis=1, inplace=True)
df.drop('Is_Foreigner', axis=1, inplace=True)

In [7]:
df.loc[df["Gross_HHLD_Income"].isnull(),"Gross_HHLD_Income"] = np.nanmedian(df.Gross_HHLD_Income)
df.loc[df["Cust_Segment"].isnull(),"Cust_Segment"] = '02 - PARTICULARES'
df.loc[df["Join_Channel"].isnull(),"Join_Channel"] = 'KHE'
df.loc[df["Cust_Relship_Type_Mth_Beg"].isnull(),"Cust_Relship_Type_Mth_Beg"] = 'I'
df.loc[df["Cust_Province_Cd"].isnull(),"Cust_Province_Cd"] = 28.0
df.loc[df["Cust_Province_Name"].isnull(),"Cust_Province_Name"] = 'MADRID'
df.loc[df["Cust_Sex"].isnull(),"Cust_Sex"] = 'V'

In [8]:
numeric_column = ['Age', 'Cust_Province_Cd', 'Cust_Active_Status', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']

In [9]:
df.columns

Index(['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Cd', 'Cust_Province_Name', 'Cust_Active_Status',
       'Gross_HHLD_Income', 'Cust_Segment', 'Savings_Acct', 'Guarantees',
       'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct',
       'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct',
       'Short_Term_Deposits', 'Med_Term_Deposits', 'Long_Term_Deposits',
       'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans', 'Taxes',
       'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2',
       'Direct_Debit', 'Monthly'],
      dtype='object')

In [10]:
df.Is_New_Cust.value_counts(dropna=False)

0.0    965500
1.0     34500
Name: Is_New_Cust, dtype: int64

In [11]:
df.Emp_Status.value_counts(dropna=False)

N    999480
B       219
F       152
A       148
S         1
Name: Emp_Status, dtype: int64

In [12]:
df.Cust_Sex.value_counts(dropna=False)

V    537136
H    462864
Name: Cust_Sex, dtype: int64

In [13]:
df.Cust_Relship_Type_Mth_Beg.value_counts(dropna=False)

I    584446
A    415130
P       343
R        81
Name: Cust_Relship_Type_Mth_Beg, dtype: int64

In [14]:
df.Residence_Ctry.value_counts(dropna=False)

S    995999
N      4001
Name: Residence_Ctry, dtype: int64

In [15]:
df.Join_Channel.value_counts(dropna=False)

KHE    312493
KFC    209783
KAT    208862
KHQ     75118
KHM     31202
        ...  
KDL         1
KDI         1
KHR         1
KDB         1
KGN         1
Name: Join_Channel, Length: 162, dtype: int64

In [16]:
df.Cust_Province_Name.value_counts(dropna=False)

MADRID                    315332
BARCELONA                  94747
VALENCIA                   53250
SEVILLA                    44053
CORUÑA, A                  32303
MURCIA                     31784
MALAGA                     26331
ZARAGOZA                   24702
ALICANTE                   24132
CADIZ                      21459
PONTEVEDRA                 21456
ASTURIAS                   20052
VALLADOLID                 17570
PALMAS, LAS                17475
BADAJOZ                    14439
TOLEDO                     14073
GRANADA                    13688
BIZKAIA                    12828
SALAMANCA                  12190
CANTABRIA                  11421
CORDOBA                    10470
BALEARS, ILLES              9701
CACERES                     9591
CIUDAD REAL                 9100
HUELVA                      8919
ALBACETE                    8734
TARRAGONA                   8081
CASTELLON                   7922
BURGOS                      7127
GIRONA                      6802
LUGO      

In [17]:
df.Cust_Segment.value_counts(dropna=False)

02 - PARTICULARES     563398
03 - UNIVERSITARIO    400674
01 - TOP               35928
Name: Cust_Segment, dtype: int64

In [18]:
from sklearn import preprocessing

scale_list = []
normalise_list = []
data_skew = df.skew()

for i in numeric_column:
    if data_skew[i] >= -0.5 and data_skew[i] <= 0.5:
        scale_list.append(i)
    else:
        normalise_list.append(i)
print(scale_list)
print(normalise_list)

['Cust_Province_Cd', 'Cust_Active_Status']
['Age', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']


C:\Users\Fahad.A\AppData\Local\Temp\ipykernel_8228\252316343.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_skew = df.skew()


In [19]:
features = df[scale_list]
scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df[scale_list] = features
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Cd,Cust_Province_Name,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Monthly
0,N,V,22,0.0,28,I,S,KHE,1.511819,VALENCIA,...,0,0,0,0,0,0,0.0,0.0,0,4
1,N,H,27,0.0,28,A,S,KHE,0.113210,MADRID,...,0,0,0,0,0,0,0.0,0.0,1,4
2,N,H,26,0.0,28,A,S,KFC,0.113210,MADRID,...,0,0,0,0,0,0,1.0,1.0,0,4
3,N,H,26,0.0,28,I,S,KHE,0.113210,MADRID,...,0,0,0,0,0,0,0.0,0.0,0,4
4,N,H,29,0.0,28,I,S,KHE,-0.663795,GRANADA,...,0,0,0,0,0,0,0.0,0.0,0,4


In [20]:
features = df[normalise_list]
scaler_1 = preprocessing.MinMaxScaler().fit(features.values)
features = scaler_1.transform(features.values)
df[normalise_list] = features
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Cd,Cust_Province_Name,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Monthly
0,N,V,0.123457,0.0,0.999772,I,S,KHE,1.511819,VALENCIA,...,0,0,0,0,0,0,0.0,0.0,0,0.0
1,N,H,0.154321,0.0,0.999772,A,S,KHE,0.113210,MADRID,...,0,0,0,0,0,0,0.0,0.0,1,0.0
2,N,H,0.148148,0.0,0.999772,A,S,KFC,0.113210,MADRID,...,0,0,0,0,0,0,1.0,1.0,0,0.0
3,N,H,0.148148,0.0,0.999772,I,S,KHE,0.113210,MADRID,...,0,0,0,0,0,0,0.0,0.0,0,0.0
4,N,H,0.166667,0.0,0.999772,I,S,KHE,-0.663795,GRANADA,...,0,0,0,0,0,0,0.0,0.0,0,0.0


In [21]:
encoder_list = ['Emp_Status', 'Cust_Sex', 'Cust_Relship_Type_Mth_Beg', 'Join_Channel', 'Cust_Segment', 'Residence_Ctry', 'Cust_Province_Name']

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.Emp_Status) 
df.Emp_Status = le.transform(df.Emp_Status)
le0 = LabelEncoder()
le0.fit(df.Cust_Sex) 
df.Cust_Sex = le0.transform(df.Cust_Sex)
le2 = LabelEncoder()
le2.fit(df.Cust_Relship_Type_Mth_Beg) 
df.Cust_Relship_Type_Mth_Beg = le2.transform(df.Cust_Relship_Type_Mth_Beg)
le3 = LabelEncoder()
le3.fit(df.Join_Channel) 
df.Join_Channel = le3.transform(df.Join_Channel)
le4 = LabelEncoder()
le4.fit(df.Cust_Segment) 
df.Cust_Segment = le4.transform(df.Cust_Segment)
le5 = LabelEncoder()
le5.fit(df.Residence_Ctry) 
df.Residence_Ctry = le5.transform(df.Residence_Ctry)
le7 = LabelEncoder()
le7.fit(df.Cust_Province_Name) 
df.Cust_Province_Name = le7.transform(df.Cust_Province_Name)

[le,le0,le2,le3,le4,le5,le7,scaler_1,scaler]

In [23]:
df.columns

Index(['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Cd', 'Cust_Province_Name', 'Cust_Active_Status',
       'Gross_HHLD_Income', 'Cust_Segment', 'Savings_Acct', 'Guarantees',
       'Cur_Acct', 'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct',
       'Mas_Particular_Acct', 'Particular_Acct', 'Particular_Plus_Acct',
       'Short_Term_Deposits', 'Med_Term_Deposits', 'Long_Term_Deposits',
       'e-Acct', 'Funds', 'Mortgage', 'Pension1', 'Loans', 'Taxes',
       'Credit_Card', 'Securities', 'Home_Acct', 'Payroll', 'Pension2',
       'Direct_Debit', 'Monthly'],
      dtype='object')

In [24]:
result = df.dtypes

print("Output:")
print(result)

Output:
Emp_Status                     int32
Cust_Sex                       int32
Age                          float64
Is_New_Cust                  float64
Cust_Since                   float64
Cust_Relship_Type_Mth_Beg      int32
Residence_Ctry                 int32
Join_Channel                   int32
Cust_Province_Cd             float64
Cust_Province_Name             int32
Cust_Active_Status           float64
Gross_HHLD_Income            float64
Cust_Segment                   int32
Savings_Acct                   int64
Guarantees                     int64
Cur_Acct                       int64
Derivative_Acct                int64
Payroll_Acct                   int64
Junior_Acct                    int64
Mas_Particular_Acct            int64
Particular_Acct                int64
Particular_Plus_Acct           int64
Short_Term_Deposits            int64
Med_Term_Deposits              int64
Long_Term_Deposits             int64
e-Acct                         int64
Funds                         

In [25]:
x_col = ['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Cd', 'Cust_Province_Name', 'Cust_Active_Status',
       'Gross_HHLD_Income', 'Cust_Segment', 'Monthly']
y_col = ['Derivative_Acct',
       'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',
       'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits',
       'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1',
       'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll',
       'Pension2', 'Direct_Debit', 'Savings_Acct', 'Guarantees', 'Cur_Acct']

In [26]:
X = df.drop(y_col,axis=1)               
y = df.drop(x_col,axis=1) 

In [27]:
X.columns

Index(['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Cd', 'Cust_Province_Name', 'Cust_Active_Status',
       'Gross_HHLD_Income', 'Cust_Segment', 'Monthly'],
      dtype='object')

In [28]:
y.columns

Index(['Savings_Acct', 'Guarantees', 'Cur_Acct', 'Derivative_Acct',
       'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',
       'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits',
       'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1',
       'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll',
       'Pension2', 'Direct_Debit'],
      dtype='object')

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [30]:
from keras.layers import LayerNormalization
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

In [31]:
model = Sequential()
model.add(Dense(128, activation='relu',input_dim=14)) 
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.2))
model.add(LayerNormalization(axis=-1 , center=True , scale=True))
model.add(Dense(24, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1920      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 48)                12336     
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
                                                                 
 layer_normalization (LayerN  (None, 48)               96        
 ormalization)                                                   
                                                        

In [32]:
model.fit(X_train,y_train, epochs=10, batch_size=32, validation_split=.2, verbose=2)

Epoch 1/10
18750/18750 - 63s - loss: 2.6103 - categorical_accuracy: 0.6106 - val_loss: 2.4813 - val_categorical_accuracy: 0.6094 - 63s/epoch - 3ms/step
Epoch 2/10
18750/18750 - 53s - loss: 2.4543 - categorical_accuracy: 0.6111 - val_loss: 2.4749 - val_categorical_accuracy: 0.6094 - 53s/epoch - 3ms/step
Epoch 3/10
18750/18750 - 49s - loss: 2.4206 - categorical_accuracy: 0.6111 - val_loss: 2.3764 - val_categorical_accuracy: 0.6094 - 49s/epoch - 3ms/step
Epoch 4/10
18750/18750 - 51s - loss: 2.4122 - categorical_accuracy: 0.6111 - val_loss: 2.3863 - val_categorical_accuracy: 0.6094 - 51s/epoch - 3ms/step
Epoch 5/10
18750/18750 - 53s - loss: 2.4113 - categorical_accuracy: 0.6111 - val_loss: 2.3804 - val_categorical_accuracy: 0.6094 - 53s/epoch - 3ms/step
Epoch 6/10
18750/18750 - 54s - loss: 2.3994 - categorical_accuracy: 0.6111 - val_loss: 2.3809 - val_categorical_accuracy: 0.6094 - 54s/epoch - 3ms/step
Epoch 7/10
18750/18750 - 63s - loss: 2.4167 - categorical_accuracy: 0.6111 - val_loss: 2

In [33]:
prediction=model.predict(X_test)
print("prediction\n", prediction)
print("\nPrediction Shape-",prediction.shape)

7813/7813 [==============================] - 18s 2ms/step
prediction
 [[  0.         0.       184.3447   ...   0.         0.         0.      ]
 [  0.         0.       123.63893  ...  31.210033  33.07903   70.47592 ]
 [  0.         0.       233.2133   ...   8.150521   7.213275  24.382687]
 ...
 [  0.         0.       181.52219  ...   0.         0.         0.      ]
 [  0.         0.       199.40733  ...   0.         0.         0.      ]
 [  0.         0.       120.044334 ...  27.4553    27.116459  63.584633]]

Prediction Shape- (250000, 24)


In [34]:
prediction[10]

array([  0.       ,   0.       , 198.19577  ,   0.       ,   0.       ,
         2.3556275,   2.5658185,  63.87725  ,  12.401817 ,   0.       ,
         0.       ,   0.       ,   8.428133 ,   0.       ,   0.       ,
         0.       ,   0.       ,   7.0579095,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ], dtype=float32)

In [35]:
ndarray = y_test.to_numpy()

In [36]:
ndarray

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [37]:
import copy
pred = copy.deepcopy(prediction)

In [38]:
for row in pred:
    for i in np.arange(np.size(row)):
        if row[i] >0.0 :
            row[i] = 1

In [39]:
matches = 0
miss = 0

In [40]:
for i in range(250000):
    for j in range(24):
        if ndarray[i][j] == 1.0:
            if ndarray[i][j] == pred[i][j]:
                matches = matches + 1
            else:
                miss = miss + 1

In [41]:
print(matches)
miss

322829


3528

In [43]:
def changeArr(input1):
    newArray = input1.copy() 
    newArray.sort()
     
    ranks = {}
     
    rank = 0
     
    for index in range(len(newArray)):
        element = newArray[index]
     
        if element not in ranks:
            ranks[element] = rank
            rank += 1
         
    
    for index in range(len(input1)):
        element = input1[index]
        input1[index] = ranks[input1[index]]
    return input1

In [50]:
pred7 = prediction[249990:250000]
lst = pred7.tolist()


In [51]:
val = []
for row in lst:
    val.append(changeArr(row))
print(val)

[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 9, 0, 7, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 2, 0, 0, 6, 5, 8], [0, 0, 7, 0, 0, 2, 1, 6, 5, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 7, 0, 0, 1, 2, 6, 5, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 16, 0, 14, 7, 4, 8, 5, 0, 0, 6, 13, 2, 0, 1, 0, 10, 9, 3, 0, 12, 11, 15], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 7, 0, 0, 1, 2, 6, 5, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 7, 0, 0, 2, 1, 6, 4, 0, 0, 0, 5, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 16, 0, 14, 5, 4, 8, 6, 0, 0, 7, 13, 2, 0, 1, 0, 10, 9, 3, 0, 12, 11, 15]]


In [52]:
y_col = ['Derivative_Acct',
       'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',
       'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits',
       'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1',
       'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll',
       'Pension2', 'Direct_Debit', 'Savings_Acct', 'Guarantees', 'Cur_Acct']

In [53]:
for row in val:
    zip_iterator = zip(y_col, row)
    a_dictionary = dict(zip_iterator)
    import operator
    sorted_dict = sorted(a_dictionary.items(), \
                key = operator.itemgetter(1), reverse=True)
    
    # Printing sorted dictionary
    print("--------")
    #print(sorted_dict)
    for i in sorted_dict:
        if i[1]>0 :
            print(i[0])

--------
Junior_Acct
--------
Junior_Acct
Cur_Acct
Particular_Acct
Savings_Acct
Guarantees
Home_Acct
Pension1
Payroll
Short_Term_Deposits
--------
Junior_Acct
Med_Term_Deposits
Long_Term_Deposits
Pension1
Home_Acct
Particular_Plus_Acct
Short_Term_Deposits
--------
Junior_Acct
--------
Junior_Acct
Med_Term_Deposits
Long_Term_Deposits
Pension1
Home_Acct
Short_Term_Deposits
Particular_Plus_Acct
--------
Junior_Acct
Cur_Acct
Particular_Acct
Pension1
Savings_Acct
Guarantees
Home_Acct
Payroll
Med_Term_Deposits
Particular_Plus_Acct
Mortgage
Long_Term_Deposits
Short_Term_Deposits
Pension2
Loans
Credit_Card
--------
Junior_Acct
--------
Junior_Acct
Med_Term_Deposits
Long_Term_Deposits
Pension1
Home_Acct
Short_Term_Deposits
Particular_Plus_Acct
--------
Junior_Acct
Med_Term_Deposits
Pension1
Long_Term_Deposits
Home_Acct
Particular_Plus_Acct
Short_Term_Deposits
--------
Junior_Acct
Cur_Acct
Particular_Acct
Pension1
Savings_Acct
Guarantees
Home_Acct
Payroll
Med_Term_Deposits
Mortgage
Long_Term_Dep

In [51]:
dff.loc[750001]

Status_Dt                           2016-05-28
Cust_ID                                1473262
Emp_Status                                   N
Cust_Ctry                                   ES
Cust_Sex                                     H
Age                                         57
Join_Dt                             2015-10-08
Is_New_Cust                                0.0
Cust_Since                                   7
Relship_Status                             1.0
Lst_Dt_Primary_Cust                        NaN
Cust_Type_Mth_Beg                            1
Cust_Relship_Type_Mth_Beg                    A
Residence_Ctry                               S
Is_Foreigner                                 N
Is_Spouse_Emp                              NaN
Join_Channel                               KHN
Deceased_Flg                                 N
Address_Type                               1.0
Cust_Province_Cd                          46.0
Cust_Province_Name                    VALENCIA
Cust_Active_S

In [48]:
X_test.loc[54510]

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Cd,Cust_Province_Name,Cust_Active_Status,Gross_HHLD_Income,Cust_Segment,Monthly
157105,3,1,0.203704,0.0,0.999868,1,1,20,-0.042191,39,-0.849968,0.006063,1,1.0
374554,3,0,0.253086,0.0,0.999991,0,1,24,0.113210,30,1.176515,0.009077,1,1.0
688694,3,1,0.283951,1.0,0.999744,1,1,150,-0.974597,17,1.176515,0.003429,1,1.0
265381,3,1,0.277778,0.0,0.999838,0,1,122,-1.673902,5,1.176515,0.002497,1,1.0
955415,3,0,0.123457,0.0,0.999778,1,1,150,-0.896897,18,1.176515,0.001856,2,1.0


In [34]:
x_col

['Emp_Status',
 'Cust_Sex',
 'Age',
 'Is_New_Cust',
 'Cust_Since',
 'Cust_Relship_Type_Mth_Beg',
 'Residence_Ctry',
 'Join_Channel',
 'Cust_Province_Cd',
 'Cust_Province_Name',
 'Cust_Active_Status',
 'Gross_HHLD_Income',
 'Cust_Segment',
 'Monthly']

In [35]:
encoder_list

['Emp_Status',
 'Cust_Sex',
 'Cust_Relship_Type_Mth_Beg',
 'Join_Channel',
 'Cust_Segment',
 'Residence_Ctry',
 'Cust_Province_Name']

In [36]:
scale_list

['Cust_Province_Cd', 'Cust_Active_Status']

In [37]:
normalise_list

['Age', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']

In [49]:
dff= pd.read_csv('sliced.csv')
dff.loc[0]   #7830  8.19 28

C:\Users\Fahad.A\AppData\Local\Temp\ipykernel_12516\59840115.py:1: DtypeWarning: Columns (10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dff= pd.read_csv('sliced.csv')


Status_Dt                            2016-04-28
Cust_ID                                 1229082
Emp_Status                                    N
Cust_Ctry                                    ES
Cust_Sex                                      V
Age                                          22
Join_Dt                              2013-12-04
Is_New_Cust                                 0.0
Cust_Since                                   28
Relship_Status                              1.0
Lst_Dt_Primary_Cust                         NaN
Cust_Type_Mth_Beg                           1.0
Cust_Relship_Type_Mth_Beg                     I
Residence_Ctry                                S
Is_Foreigner                                  N
Is_Spouse_Emp                               NaN
Join_Channel                                KHE
Deceased_Flg                                  N
Address_Type                                1.0
Cust_Province_Cd                           46.0
Cust_Province_Name                     V

In [6]:
dff.loc[54510]

Status_Dt                           2016-04-28
Cust_ID                                1303924
Emp_Status                                   N
Cust_Ctry                                   ES
Cust_Sex                                     H
Age                                         57
Join_Dt                             2014-09-09
Is_New_Cust                                0.0
Cust_Since                                  19
Relship_Status                             1.0
Lst_Dt_Primary_Cust                        NaN
Cust_Type_Mth_Beg                          1.0
Cust_Relship_Type_Mth_Beg                    I
Residence_Ctry                               S
Is_Foreigner                                 N
Is_Spouse_Emp                              NaN
Join_Channel                               KFC
Deceased_Flg                                 N
Address_Type                               1.0
Cust_Province_Cd                          31.0
Cust_Province_Name                     NAVARRA
Cust_Active_S

In [5]:
dff.loc[124540]

Status_Dt                           2016-05-28
Cust_ID                                 697619
Emp_Status                                   N
Cust_Ctry                                   ES
Cust_Sex                                     V
Age                                         52
Join_Dt                             2007-06-16
Is_New_Cust                                0.0
Cust_Since                                 107
Relship_Status                             1.0
Lst_Dt_Primary_Cust                        NaN
Cust_Type_Mth_Beg                          1.0
Cust_Relship_Type_Mth_Beg                    I
Residence_Ctry                               S
Is_Foreigner                                 N
Is_Spouse_Emp                              NaN
Join_Channel                               KAG
Deceased_Flg                                 N
Address_Type                               1.0
Cust_Province_Cd                          28.0
Cust_Province_Name                      MADRID
Cust_Active_S

In [39]:
lst = []
for i in range(len(scale_list)):
    ele = float(input())
    lst.append(ele)

In [40]:
lst = np.asarray(lst)

In [41]:
B = np.reshape(lst, (1, 2))

In [42]:
B

array([[46.,  0.]])

In [43]:
B = scaler.transform(B)

In [44]:
B

array([[ 1.51181861, -0.84996775]])

In [79]:
B[0][1]

-0.8499677542013256

In [45]:
date = input()

x = date.split('-')

month = int(x[1])
month

4

In [46]:
lst = []
for i in range(3):
    ele = float(input())
    lst.append(ele)

In [47]:
lst.append(month)
lst = np.asarray(lst)

In [48]:
lst

array([2.200000e+01, 7.830819e+04, 2.800000e+01, 4.000000e+00])

In [49]:
A = np.reshape(lst, (1, 4))

In [50]:
A

array([[2.200000e+01, 7.830819e+04, 2.800000e+01, 4.000000e+00]])

In [51]:
A = scaler_1.transform(A)

In [52]:
A

array([[0.12345679, 0.00266864, 0.99977206, 0.        ]])

In [81]:
A[0][2]

0.0

In [ ]:
lst = []
for i in range(len(scale_list)):
    ele = float(input())
    lst.append(ele)
lst = np.asarray(lst)
B = np.reshape(lst, (1, 2))
B = scaler.transform(B)
date = input()

x = date.split('-')

month = int(x[1])
month

lst = []
for i in range(3):
    ele = float(input())
    lst.append(ele)

lst.append(month)
lst = np.asarray(lst)
A = np.reshape(lst, (1, 4))
A = scaler_1.transform(A)

In [53]:
lst = []
Emp_Stat = input()
lst.append(Emp_Stat)
Emp_Stat = le.transform(lst)

In [54]:
Emp_Sta = Emp_Stat[0]
Emp_Sta

3

In [ ]:
lst = []
Emp_Stat = input()
lst.append(Emp_Stat)
Emp_Stat = le.transform(lst)
Emp_Sta = Emp_Stat[0]
lst = []
CSex = input()
lst.append(CSex)
CSex = le0.transform(lst)
Sex = CSex[0]
lst = []
Cust_Relship = input()
lst.append(Cust_Relship)
Cust_Relship = le2.transform(lst)
Cust_Rel = Cust_Relship[0]
lst = []
Join_Chan = input()
lst.append(Join_Chan)
Join_Chan = le3.transform(lst)
Join_Ch = Join_Chan[0]
lst = []
Cust_Seg = input()
lst.append(Cust_Seg)
Cust_Seg = le4.transform(lst)
Seg = Cust_Seg[0]
lst = []
Residence_Ctr = input()
lst.append(Residence_Ctr)
Residence_Ctr = le5.transform(lst)
Country = Residence_Ctr[0]
lst = []
Cust_Province = input()
lst.append(Cust_Province)
Cust_Province = le7.transform(lst)
Province = Cust_Province[0]


In [55]:
lst = []
CSex = input()
lst.append(CSex)
CSex = le0.transform(lst)
Sex = CSex[0]
Sex

1

In [56]:
lst = []
Cust_Relship = input()
lst.append(Cust_Relship)
Cust_Relship = le2.transform(lst)
Cust_Rel = Cust_Relship[0]
Cust_Rel

1

In [57]:
lst = []
Join_Chan = input()
lst.append(Join_Chan)
Join_Chan = le3.transform(lst)
Join_Ch = Join_Chan[0]
Join_Ch

150

In [58]:
lst = []
Cust_Seg = input()
lst.append(Cust_Seg)
Cust_Seg = le4.transform(lst)
Seg = Cust_Seg[0]
Seg

2

In [59]:
lst = []
Residence_Ctr = input()
lst.append(Residence_Ctr)
Residence_Ctr = le5.transform(lst)
Country = Residence_Ctr[0]
Country

1

In [60]:
lst = []
Cust_Province = input()
lst.append(Cust_Province)
Cust_Province = le7.transform(lst)
Province = Cust_Province[0]
Province

48

['Emp_Status',
 'Cust_Sex',
 'Age',
 'Is_New_Cust',
 'Cust_Since',
 'Cust_Relship_Type_Mth_Beg',
 'Residence_Ctry',
 'Join_Channel',
 'Cust_Province_Cd',
 'Cust_Province_Name',
 'Cust_Active_Status',
 'Gross_HHLD_Income',
 'Cust_Segment',
 'Monthly']

In [61]:
Is_New = float(input())
print(Is_New)
#Cust_Sin = int(input())
#Cust_Sin

0.0


In [62]:
lst = []
lst.append(Emp_Sta)
lst.append(Sex)
lst.append(A[0][0])
lst.append(Is_New)
lst.append(A[0][2])
lst.append(Cust_Rel)
lst.append(Country)
lst.append(Join_Ch)
lst.append(B[0][0])
lst.append(Province)
lst.append(B[0][1])
lst.append(A[0][1])
lst.append(Seg)
lst.append(A[0][3])

In [63]:
lst = np.asarray(lst)

In [64]:
lst

array([ 3.00000000e+00,  1.00000000e+00,  1.23456790e-01,  0.00000000e+00,
        9.99772058e-01,  1.00000000e+00,  1.00000000e+00,  1.50000000e+02,
        1.51181861e+00,  4.80000000e+01, -8.49967754e-01,  2.66863758e-03,
        2.00000000e+00,  0.00000000e+00])

In [65]:
X_train.shape

(750000, 14)

In [66]:
lst.shape

(14,)

In [67]:
C = np.reshape(lst, (1, 14))

In [68]:
C

array([[ 3.00000000e+00,  1.00000000e+00,  1.23456790e-01,
         0.00000000e+00,  9.99772058e-01,  1.00000000e+00,
         1.00000000e+00,  1.50000000e+02,  1.51181861e+00,
         4.80000000e+01, -8.49967754e-01,  2.66863758e-03,
         2.00000000e+00,  0.00000000e+00]])

In [69]:
pred4=model.predict(C)

1/1 [==============================] - 0s 149ms/step


In [70]:
pred4

array([[  0.    ,   0.    , 139.7945,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ]],
      dtype=float32)

In [71]:
def changeArr(input1):
    newArray = input1.copy() 
    newArray.sort()
     
    ranks = {}
     
    rank = 0
     
    for index in range(len(newArray)):
        element = newArray[index]
     
        if element not in ranks:
            ranks[element] = rank
            rank += 1
         
    
    for index in range(len(input1)):
        element = input1[index]
        input1[index] = ranks[input1[index]]

pred5 = pred4[0]
changeArr(pred5)
print(pred5)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [72]:
zip_iterator = zip(y_col, pred5)
a_dictionary = dict(zip_iterator)

print(a_dictionary)

{'Derivative_Acct': 0.0, 'Payroll_Acct': 0.0, 'Junior_Acct': 1.0, 'Mas_Particular_Acct': 0.0, 'Particular_Acct': 0.0, 'Particular_Plus_Acct': 0.0, 'Short_Term_Deposits': 0.0, 'Med_Term_Deposits': 0.0, 'Long_Term_Deposits': 0.0, 'e-Acct': 0.0, 'Funds': 0.0, 'Mortgage': 0.0, 'Pension1': 0.0, 'Loans': 0.0, 'Taxes': 0.0, 'Credit_Card': 0.0, 'Securities': 0.0, 'Home_Acct': 0.0, 'Payroll': 0.0, 'Pension2': 0.0, 'Direct_Debit': 0.0, 'Savings_Acct': 0.0, 'Guarantees': 0.0, 'Cur_Acct': 0.0}


In [73]:
import operator

In [74]:
import operator
sorted_dict = sorted(a_dictionary.items(), \
             key = operator.itemgetter(1), reverse=True)
 
# Printing sorted dictionary
print("Sorted dictionary is :")
print(sorted_dict)

Sorted dictionary is :
[('Junior_Acct', 1.0), ('Derivative_Acct', 0.0), ('Payroll_Acct', 0.0), ('Mas_Particular_Acct', 0.0), ('Particular_Acct', 0.0), ('Particular_Plus_Acct', 0.0), ('Short_Term_Deposits', 0.0), ('Med_Term_Deposits', 0.0), ('Long_Term_Deposits', 0.0), ('e-Acct', 0.0), ('Funds', 0.0), ('Mortgage', 0.0), ('Pension1', 0.0), ('Loans', 0.0), ('Taxes', 0.0), ('Credit_Card', 0.0), ('Securities', 0.0), ('Home_Acct', 0.0), ('Payroll', 0.0), ('Pension2', 0.0), ('Direct_Debit', 0.0), ('Savings_Acct', 0.0), ('Guarantees', 0.0), ('Cur_Acct', 0.0)]


In [75]:
for i in sorted_dict:
    if i[1]>0 :
        print(i[0])

Junior_Acct


In [76]:
pickle.dump(scaler, open('Scaled.pkl', 'wb'))
pickle.dump(scaler_1, open('Minmaxed.pkl', 'wb'))
pickle.dump(le, open('Encoded.pkl', 'wb'))
pickle.dump(le0, open('Encoded0.pkl', 'wb'))
pickle.dump(le2, open('Encoded2.pkl', 'wb'))
pickle.dump(le3, open('Encoded3.pkl', 'wb'))
pickle.dump(le4, open('Encoded4.pkl', 'wb'))
pickle.dump(le5, open('Encoded5.pkl', 'wb'))
pickle.dump(le7, open('Encoded7.pkl', 'wb'))

In [77]:
model.save('Recommendation.h5')
print('Model Saved!')

Model Saved!


In [3]:
scaled_model = pickle.load(open('Scaled.pkl', 'rb'))
minmaxed_model = pickle.load(open('Minmaxed.pkl', 'rb'))
Emp_Stat_model = pickle.load(open('Encoded.pkl', 'rb'))
Sex_model = pickle.load(open('Encoded0.pkl', 'rb'))
Cust_Rel_model = pickle.load(open('Encoded2.pkl', 'rb'))
Join_Ch_pickled_model = pickle.load(open('Encoded3.pkl', 'rb'))
Seg_pickled_model = pickle.load(open('Encoded4.pkl', 'rb'))
Country_pickled_model = pickle.load(open('Encoded5.pkl', 'rb'))
Provincename_pickled_model = pickle.load(open('Encoded7.pkl', 'rb'))

In [6]:
lst = []
Emp_Stat = input()
lst.append(Emp_Stat)
Emp_Stat = Emp_Stat_model.transform(lst)
Emp_Sta = Emp_Stat[0]
lst = []
CSex = input()
lst.append(CSex)
CSex = Sex_model.transform(lst)
Sex = CSex[0]
lst = []
Cust_Relship = input()
lst.append(Cust_Relship)
Cust_Relship = Cust_Rel_model.transform(lst)
Cust_Rel = Cust_Relship[0]
lst = []
Join_Chan = input()
lst.append(Join_Chan)
Join_Chan = Join_Ch_pickled_model.transform(lst)
Join_Ch = Join_Chan[0]
lst = []
Cust_Seg = input()
lst.append(Cust_Seg)
Cust_Seg = Seg_pickled_model.transform(lst)
Seg = Cust_Seg[0]
lst = []
Residence_Ctr = input()
lst.append(Residence_Ctr)
Residence_Ctr = Country_pickled_model.transform(lst)
Country = Residence_Ctr[0]
lst = []
Cust_Province = input()
lst.append(Cust_Province)
Cust_Province = Provincename_pickled_model.transform(lst)
Province = Cust_Province[0]

In [7]:
#['Cust_Province_Cd', 'Cust_Active_Status']
#['Age', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']

In [9]:
lst = []
for i in range(2):
    ele = float(input())
    lst.append(ele)
lst = np.asarray(lst)
B = np.reshape(lst, (1, 2))
B = scaled_model.transform(B)
date = input()

x = date.split('-')

month = int(x[1])
month

lst = []
for i in range(3):
    ele = float(input())
    lst.append(ele)

lst.append(month)
lst = np.asarray(lst)
A = np.reshape(lst, (1, 4))
A = minmaxed_model.transform(A)

In [10]:
Is_New_cust = float(input())

In [11]:
lst = []
lst.append(Emp_Sta)
lst.append(Sex)
lst.append(A[0][0])
lst.append(Is_New_cust)
lst.append(A[0][2])
lst.append(Cust_Rel)
lst.append(Country)
lst.append(Join_Ch)
lst.append(B[0][0])
lst.append(Province)
lst.append(B[0][1])
lst.append(A[0][1])
lst.append(Seg)
lst.append(A[0][3])
lst = np.asarray(lst)
C = np.reshape(lst, (1, 14))

In [12]:
from keras.models import load_model
savedModel=load_model('Recommendation.h5')
savedModel.summary()
predictions = savedModel.predict(C)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1920      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 48)                12336     
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
                                                                 
 layer_normalization (LayerN  (None, 48)               96        
 ormalization)                                                   
                                                        

In [13]:
def changeArr(input1):
    newArray = input1.copy() 
    newArray.sort()
     
    ranks = {}
     
    rank = 0
     
    for index in range(len(newArray)):
        element = newArray[index]
     
        if element not in ranks:
            ranks[element] = rank
            rank += 1
         
    
    for index in range(len(input1)):
        element = input1[index]
        input1[index] = ranks[input1[index]]



In [14]:
pred6 = predictions[0]
changeArr(pred6)
print(pred6)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
y_col = ['Derivative_Acct',
       'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',
       'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits',
       'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1',
       'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll',
       'Pension2', 'Direct_Debit', 'Savings_Acct', 'Guarantees', 'Cur_Acct']

In [16]:
zip_iterator = zip(y_col, pred6)
a_dictionary = dict(zip_iterator)
import operator
sorted_dict = sorted(a_dictionary.items(), \
             key = operator.itemgetter(1), reverse=True)
 
# Printing sorted dictionary
print("Sorted dictionary is :")
print(sorted_dict)
for i in sorted_dict:
    if i[1]>0 :
        print(i[0])

Sorted dictionary is :
[('Junior_Acct', 1.0), ('Derivative_Acct', 0.0), ('Payroll_Acct', 0.0), ('Mas_Particular_Acct', 0.0), ('Particular_Acct', 0.0), ('Particular_Plus_Acct', 0.0), ('Short_Term_Deposits', 0.0), ('Med_Term_Deposits', 0.0), ('Long_Term_Deposits', 0.0), ('e-Acct', 0.0), ('Funds', 0.0), ('Mortgage', 0.0), ('Pension1', 0.0), ('Loans', 0.0), ('Taxes', 0.0), ('Credit_Card', 0.0), ('Securities', 0.0), ('Home_Acct', 0.0), ('Payroll', 0.0), ('Pension2', 0.0), ('Direct_Debit', 0.0), ('Savings_Acct', 0.0), ('Guarantees', 0.0), ('Cur_Acct', 0.0)]
Junior_Acct
